# Поддержка протокола контекста модели (MCP) в Azure AI Agents - Python

Этот ноутбук демонстрирует, как использовать Azure AI Agents с инструментами протокола контекста модели (MCP) в Python. Он показывает, как создать интеллектуального агента, который может использовать внешние MCP-серверы (например, Microsoft Learn) для расширенных возможностей с использованием аутентификации без ключей.


## Установка необходимых пакетов Python

Сначала необходимо установить необходимые пакеты Python:
- **azure-ai-projects**: Основной SDK для Azure AI Projects
- **azure-ai-agents**: SDK Azure AI Agents для создания и управления агентами
- **azure-identity**: Обеспечивает аутентификацию без ключей с использованием DefaultAzureCredential
- **mcp**: Реализация протокола Model Context Protocol для Python


## Преимущества аутентификации без ключей

Этот ноутбук демонстрирует **аутентификацию без ключей**, которая предоставляет несколько преимуществ:
- ✅ **Нет необходимости управлять API-ключами** - Использует аутентификацию на основе идентификации Azure
- ✅ **Повышенная безопасность** - Секреты не хранятся в коде или файлах конфигурации
- ✅ **Автоматическая ротация учетных данных** - Azure управляет жизненным циклом учетных данных
- ✅ **Контроль доступа на основе ролей** - Использует Azure RBAC для детализированного управления разрешениями
- ✅ **Поддержка нескольких сред** - Бесшовно работает как в среде разработки, так и в производственной среде

`DefaultAzureCredential` автоматически выбирает лучший доступный источник учетных данных:
1. **Управляемая идентификация** (при запуске в Azure)
2. Учетные данные **Azure CLI** (во время локальной разработки)
3. Учетные данные **Visual Studio**
4. **Переменные окружения** (если настроены)
5. Аутентификация через **интерактивный браузер** (в качестве резервного варианта)


## Настройка аутентификации без ключей

**Требования для аутентификации без ключей:**

### Для локальной разработки:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Для сред Azure:
- Включите **Системно назначенную управляемую идентичность** на вашем ресурсе Azure
- Назначьте соответствующие **RBAC роли** для управляемой идентичности:
  - `Cognitive Services OpenAI User` для доступа к Azure OpenAI
  - `AI Developer` для доступа к проектам Azure AI

### Переменные окружения (опционально):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Никаких API-ключей или строк подключения не требуется!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Импорт необходимых библиотек

Импортируйте необходимые модули Python:
- **os, time**: Стандартные библиотеки Python для работы с переменными окружения и задержками
- **AIProjectClient**: Основной клиент для Azure AI Projects
- **DefaultAzureCredential**: Безключевая аутентификация для сервисов Azure
- **Классы, связанные с MCP**: Для создания и управления инструментами MCP и обработки утверждений


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Настройка параметров сервера MCP

Настройте конфигурацию сервера MCP с использованием переменных окружения с резервными значениями по умолчанию:
- **MCP_SERVER_URL**: URL сервера MCP (по умолчанию используется API Microsoft Learn)
- **MCP_SERVER_LABEL**: Метка для идентификации сервера MCP (по умолчанию "mslearn")

Этот подход обеспечивает гибкую настройку для различных сред.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Создание клиента Azure AI Project (Аутентификация без ключей)

Инициализируйте клиент Azure AI Project с использованием **аутентификации без ключей**:
- **endpoint**: URL конечной точки проекта Azure AI Foundry
- **credential**: Использует `DefaultAzureCredential()` для безопасной аутентификации без ключей
- **Ключи API не требуются**: Автоматически обнаруживает и использует наиболее подходящие доступные учетные данные

**Процесс аутентификации:**
1. Проверяет наличие управляемой идентичности (в средах Azure)
2. Переходит к учетным данным Azure CLI (для локальной разработки)
3. Использует другие доступные источники учетных данных при необходимости

Этот подход устраняет необходимость управления ключами API или строками подключения в вашем коде.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Создание определения инструмента MCP

Создайте инструмент MCP, который подключается к серверу Microsoft Learn MCP:
- **server_label**: Идентификатор сервера MCP
- **server_url**: URL-адрес конечной точки сервера MCP
- **allowed_tools**: Необязательный список для ограничения использования инструментов (пустой список позволяет использовать все инструменты)

Этот инструмент позволит агенту получать доступ к документации и ресурсам Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Создание агента и выполнение диалога (безключевой процесс)

Этот подробный раздел демонстрирует полный **процесс работы агента без ключей**:

1. **Создание AI-агента**: Настройте агента с использованием модели GPT-4.1 nano и инструментов MCP  
2. **Создание потока**: Установите поток общения для ведения диалога  
3. **Отправка сообщения**: Спросите агента о различиях между Azure OpenAI и OpenAI  
4. **Обработка одобрений инструментов**: Автоматически одобряйте вызовы инструментов MCP, если это необходимо  
5. **Мониторинг выполнения**: Отслеживайте прогресс агента и выполняйте необходимые действия  
6. **Отображение результатов**: Покажите детали диалога и использования инструментов  

**Особенности безключевого подхода:**
- ✅ **Отсутствие жестко прописанных секретов** - Вся аутентификация осуществляется через Azure Identity  
- ✅ **Безопасность по умолчанию** - Используется управление доступом на основе ролей  
- ✅ **Упрощенное развертывание** - Нет необходимости в управлении учетными данными  
- ✅ **Удобство аудита** - Весь доступ отслеживается через Azure Identity  

Агент будет использовать инструменты MCP для доступа к ресурсам Microsoft Learn с полной безопасностью и без необходимости управления API-ключами.  


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
